# 00_ETL_modelo

## 01_import_bibliotecas

In [1]:
import pandas as pd
import json
from pandas import json_normalize
import requests

## 02_import_dados

In [2]:
# import de dados da decision

vagas = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/vagas.json"
prospects = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/prospects.json"
applicants = "https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/applicants.json"

In [3]:
# lê os dados do github

# vagas
response_vagas = requests.get(vagas)
data_vagas = response_vagas.json()

# prospects
response_prospects = requests.get(prospects)
data_prospects = response_prospects.json()

# Aplicants
response_applicants = requests.get(applicants)
data_applicants = response_applicants.json()

## 03_tratamento_json

In [4]:
# Trata Vagas

records_vagas = [dict(id_vaga=k, **v) for k, v in data_vagas.items()]

df_vagas = pd.json_normalize(records_vagas, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_vagas = [
    'informacoes_basicas_',
    'perfil_vaga_',
    'beneficios_'
]

for prefixo in prefixos_para_remover_vagas:
    df_vagas.columns = df_vagas.columns.str.replace(f'^{prefixo}', '', regex=True)

In [5]:
# Trata Prospects

records_prospects = [dict(id_vaga=k, **v) for k, v in data_prospects.items()]

df_prospects = pd.json_normalize(
    records_prospects,
    record_path='prospects',
    meta=['id_vaga', 'titulo', 'modalidade'],
    sep='_'
)

# Remove os prefixos das colunas
prefixos_para_remover_prospects = [
'prospects_',  
]

for prefixo in prefixos_para_remover_prospects:
    df_prospects.columns = df_prospects.columns.str.replace(f'^{prefixo}', '', regex=True)
    
df_prospects.rename(columns={'codigo': 'id_candidato'}, inplace=True)

In [6]:
# Trata Aplicants

records_applicants = [dict(id_candidato=k, **v) for k, v in data_applicants.items()]

df_applicants = pd.json_normalize(records_applicants, sep="_")

# Remove os prefixos das colunas
prefixos_para_remover_applicants = [
    'infos_basicas_',
    'informacoes_pessoais_',
    'informacoes_profissionais_',
    'formacao_e_idiomas_',
    'cargo_atual_'
]

for prefixo in prefixos_para_remover_applicants:
    df_applicants.columns = df_applicants.columns.str.replace(f'^{prefixo}', '', regex=True)

## 04_juntando_arquivos

In [7]:
# prospects + applicants
df_inter = df_prospects.merge(df_applicants, on="id_candidato", how="left", suffixes=('_tabelaprospect', '_tabelaapplicant'))

In [8]:
# dados + Vagas
df_geral = df_inter.merge(df_vagas, on="id_vaga", how="left", suffixes=('', '_tabelavaga'))

## 05_Salvando_base_unificada_no_git

In [11]:
Caminho = "C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5\\01_fontes\\arquivos_decision\\fontes_tratadas\\"
arquivo = "base_unificada.csv"

In [12]:
df_geral.to_csv(Caminho + arquivo, encoding ="utf8" , sep=";" ,index=False)